In [1]:
import json
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout
import random

In [2]:
# Load the intents JSON file
with open('IPC_intents.json') as file:
    data = json.load(file)

In [3]:
# Extract patterns and corresponding tags
patterns = []
tags = []
tag_responses = {}

In [4]:
for intent in data['intents']:
    for pattern in intent['patterns']:
        patterns.append(pattern)
        tags.append(intent['tag'])
    tag_responses[intent['tag']] = intent['responses']

In [5]:
# Vectorize the text data
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(patterns).toarray()

In [6]:
# Encode the labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(tags)

In [7]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
# Build a neural network model
model = Sequential()
model.add(Dense(128, input_shape=(X.shape[1],), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

In [9]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [10]:
# Train the model
model.fit(X_train, y_train, epochs=500, batch_size=8, verbose=1, validation_data=(X_test, y_test))

Epoch 1/500
540/540 [==============================] - 6s 10ms/step - loss: 7.5151 - accuracy: 0.0016 - val_loss: 7.5356 - val_accuracy: 0.0019
Epoch 2/500
540/540 [==============================] - 6s 10ms/step - loss: 7.2765 - accuracy: 0.0037 - val_loss: 7.3405 - val_accuracy: 0.0019
Epoch 3/500
540/540 [==============================] - 5s 9ms/step - loss: 6.5700 - accuracy: 0.0206 - val_loss: 6.9470 - val_accuracy: 0.0074
Epoch 4/500
540/540 [==============================] - 5s 9ms/step - loss: 5.9131 - accuracy: 0.0486 - val_loss: 6.4682 - val_accuracy: 0.0601
Epoch 5/500
540/540 [==============================] - 5s 9ms/step - loss: 5.4004 - accuracy: 0.0833 - val_loss: 6.0013 - val_accuracy: 0.1267
Epoch 6/500
540/540 [==============================] - 5s 9ms/step - loss: 4.9539 - accuracy: 0.1278 - val_loss: 5.5823 - val_accuracy: 0.2664
Epoch 7/500
540/540 [==============================] - 5s 9ms/step - loss: 4.6393 - accuracy: 0.1593 - val_loss: 5.1735 - val_accuracy: 0.35

In [11]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy * 100:.2f}%')

34/34 [==============================] - 0s 3ms/step - loss: 1.8226 - accuracy: 0.7179
Accuracy: 71.79%


In [12]:
import pickle

# Save the vectorizer and label encoder
with open('vectorizer.pkl', 'wb') as file:
    pickle.dump(vectorizer, file)

with open('label_encoder.pkl', 'wb') as file:
    pickle.dump(label_encoder, file)

# Save the responses
with open('tag_responses.pkl', 'wb') as file:
    pickle.dump(tag_responses, file)
    
# Save the model
model.save('chatbot_model.h5')

c:\Users\Sontene Uday Kiran\anaconda3\envs\chat-app\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
